In [1]:
from math import gamma, comb, factorial
from scipy.special import hyp2f1, poch
import numpy as np
from scipy.integrate import quad

In [2]:
def front_expression(gamma_v, gamma_bar, beta = 13.12, k = 2.32, m = 2.3):
    """
    Calculate the mathematical expression:
    (ρ² * z^k * m^m) / (2 * (2σ² * A₀² * h_i² * γ)^(ρ²/2) * Γ(m)) * γ^(2/2)
    
    Args:
        gamma_bar: Variable input parameter
    
    Returns:
        float: Result of the mathematical expression
    """

    # Define all constant values at the top
    rho = 3.3           # ρ (rho)
    d = 30            # d in meters
    z = 4.343/(beta*d)             # z
    K = 10
    A0 = 1            # A₀ (A naught)
    h_i = 0.4993           # h_i
    
    # Calculate individual components
    rho_squared = rho**2
    z_to_k = z**k
    m_to_m = m**m
    
    # Calculate the denominator components
    sigma_squared = 1/(1+K)         # σ^2 (sigma_squared)
    A0_squared = A0**2
    h_i_squared = h_i**2
    
    # Calculate the base of the power in denominator
    base_power = 2 * sigma_squared * A0_squared * h_i_squared * gamma_bar
    
    # Calculate the exponent for the base
    exponent = rho_squared / 2
    
    # Calculate the gamma function
    gamma_function = gamma(m)
    
    # Calculate numerator
    numerator = rho_squared * z_to_k * m_to_m * (gamma_v**exponent)
    
    # Calculate denominator
    denominator = 2 * (base_power**exponent) * gamma_function
    
    # Final result
    result = numerator / denominator
    
    return result

In [3]:
def R_mu_nu(mu, nu, x):
    """
    Compute R_ν^μ(x) function with two cases:
    - For μ ∈ ℕ⁺ (positive integers): uses the first formula with x^μ term
    - Otherwise: uses the second formula with Γ(1-μ) in denominator
    
    Parameters:
    -----------
    mu : float  
        Parameter μ
    nu : float
        Parameter ν
    x : float
        Input variable x
        
    Returns:
    --------
    float
        Value of R_ν^μ(x)
    """
    
    # Check if μ is a positive integer
    if isinstance(mu, int) and mu > 0:
        # Case 1: μ ∈ ℕ⁺ (positive integers)
        
        # Calculate the hypergeometric function arguments
        a = (nu + mu) / 2
        b = (nu + mu + 1) / 2
        c = 1 + mu
        d = (nu - mu + 1) / 2
        e = (nu - mu) / 2
        
        # Calculate the binomial-like coefficients
        coeff1 = poch(d, mu)
        coeff2 = poch(e, mu)

        # Calculate x^μ/μ term
        x_power_term = (x**mu) / mu
        
        # Calculate the hypergeometric function
        hyp_func = hyp2f1(a, b, c, x)
        
        # Combine all terms
        result = x_power_term * coeff1 * coeff2 * hyp_func
        
    else:
        # Case 2: Otherwise (μ not a positive integer)
        
        # Calculate the hypergeometric function arguments
        a = (nu - mu) / 2
        b = (nu - mu + 1) / 2
        c = 1 - mu
        
        # Calculate the hypergeometric function
        hyp_func = hyp2f1(a, b, c, x)
        
        # Calculate Γ(1-μ)
        gamma_term = gamma(1 - mu)
        
        # Combine terms
        result = (hyp_func) / gamma_term
    
    return result

In [4]:
def a_j(j, m, Delta):
    """
    Compute the coefficient a_j according to the given formula
    
    Parameters:
    j : int - first parameter
    m : int - second parameter  
    Delta : float - fourth parameter
    
    Returns:
    float - computed coefficient
    """
    
    K = 10
    result = 0.0
    result_int = 0.0
    
    # Double summation over k and l
    for k in range(j + 1):  # k from 0 to j
        binom_j_k = comb(j, k)

        for l in range(k + 1):  # l from 0 to k

            # Binomial coefficients
            binom_k_l = comb(k, l)
            
            # (Delta/2)^(2l) term
            delta_term = (Delta / 2) ** (2 * l)
            
            # Gamma function term
            gamma_term = gamma(j + m + 2*l - k)
            
            # K^(2l-k) term
            K_term = K ** (2*l - k)
            
            # (-1)^(2l-k) term
            sign_term = (-1) ** (2*l - k)
            
            # (m + K)^(-(j+m+2l-k)) term
            power_term = (m + K) ** (-(j + m + 2*l - k))
            
            # R function term
            R_term = R_mu_nu(k - 2*l ,j + m ,(K * Delta / (m + K))**2)
            
            # Combine all terms
            term = (binom_k_l * delta_term * gamma_term * K_term * 
                   sign_term * power_term * R_term)
            
            result_int += term

        result += result_int*binom_j_k
    return result

In [5]:
def GammaProd(p, x, y, X):
    """
    Exact implementation of MATLAB's GammaProd function
    
    Parameters:
    p, x, y: parameter arrays
    X: integration variable (scalar or array)
    
    Returns:
    output: gamma function product
    """
    # Handle scalar X by converting to array
    X_array = np.atleast_1d(X)
    
    if len(p) == 0:
        return np.ones_like(X_array).reshape(np.shape(X))
    
    # Create meshgrid exactly as in MATLAB
    pp, XX = np.meshgrid(p, X_array, indexing='ij')
    xx, _ = np.meshgrid(x, X_array, indexing='ij')
    
    # Calculate gamma products with exact MATLAB behavior
    gamma_args = pp + xx * XX
    gamma_vals = gammaz(gamma_args) ** y
    
    # Product along first dimension (equivalent to MATLAB's prod(..., 2))
    output = np.prod(gamma_vals, axis=0)
    
    # Reshape to match X shape
    return output.reshape(np.shape(X))


In [6]:
def gammaz(z):
    # Convert input to numpy array and preserve original shape
    z = np.asarray(z, dtype=complex)
    siz = z.shape
    z = z.flatten()
    zz = z.copy()
    f = np.zeros_like(z, dtype=complex)  # reserve space in advance
    
    # Find elements with negative real parts
    p = np.where(np.real(z) < 0)[0]
    if len(p) > 0:
        z[p] = -z[p]
    
    # 15 sig. digits for 0<=real(z)<=171
    # coeffs should sum to about g*g/2+23/24
    g = 607/128  # best results when 4<=g<=5
    c = np.array([
        0.99999999999999709182,
        57.156235665862923517,
        -59.597960355475491248,
        14.136097974741747174,
        -0.49191381609762019978,
        0.33994649984811888699e-4,
        0.46523628927048575665e-4,
        -0.98374475304879564677e-4,
        0.15808870322491248884e-3,
        -0.21026444172410488319e-3,
        0.21743961811521264320e-3,
        -0.16431810653676389022e-3,
        0.84418223983852743293e-4,
        -0.26190838401581408670e-4,
        0.36899182659531622704e-5
    ])
    
    # Num Recipes used g=5 with 7 terms
    # for a less effective approximation
    z = z - 1
    zh = z + 0.5
    zgh = zh + g
    
    # trick for avoiding FP overflow above z=141
    zp = zgh**(zh * 0.5)
    ss = np.zeros_like(z, dtype=complex)
    
    for pp in range(len(c)-2, -1, -1):  # size(c,1)-1:-1:1 equivalent
        ss = ss + c[pp+1] / (z + pp)
    
    # sqrt(2Pi)
    sq2pi = 2.5066282746310005024157652848110
    f = (sq2pi * (c[0] + ss)) * ((zp * np.exp(-zgh)) * zp)
    
    # Handle special cases
    mask = (z == -1) | (z == 0)  # z==0 | z==1 in original (z was decremented by 1)
    f[mask] = 1.0
    
    # adjust for negative real parts
    if len(p) > 0:
        f[p] = -np.pi / (zz[p] * f[p] * np.sin(np.pi * zz[p]))
    
    # adjust for negative poles
    p = np.where((np.round(zz) == zz) & (np.imag(zz) == 0) & (np.real(zz) <= 0))[0]
    if len(p) > 0:
        f[p] = np.inf
    
    f = f.reshape(siz)
    return f

In [7]:
def complex_contour_integral(func, start, end, waypoints=None, abstol=1e-5, reltol=1e-5):
    """
    Exact implementation of complex contour integration matching MATLAB's integral function
    """
    if waypoints is None:
        # Direct line integration from start to end
        def real_integrand(t):
            s = start + t * (end - start)
            ds_dt = end - start
            return np.real(func(s) * ds_dt)
        
        def imag_integrand(t):
            s = start + t * (end - start)
            ds_dt = end - start
            return np.imag(func(s) * ds_dt)
        
        real_part, _ = quad(real_integrand, 0, 1, epsabs=abstol, epsrel=reltol)
        imag_part, _ = quad(imag_integrand, 0, 1, epsabs=abstol, epsrel=reltol)
        
        return real_part + 1j * imag_part
    
    else:
        # Integration with waypoints - exact MATLAB behavior
        total_integral = 0.0 + 0.0j
        
        # First segment: start to first waypoint
        wp1 = waypoints[0]
        def real_integrand1(t):
            s = start + t * (wp1 - start)
            ds_dt = wp1 - start
            return np.real(func(s) * ds_dt)
        
        def imag_integrand1(t):
            s = start + t * (wp1 - start)
            ds_dt = wp1 - start
            return np.imag(func(s) * ds_dt)
        
        real_part1, _ = quad(real_integrand1, 0, 1, epsabs=abstol, epsrel=reltol)
        imag_part1, _ = quad(imag_integrand1, 0, 1, epsabs=abstol, epsrel=reltol)
        total_integral += real_part1 + 1j * imag_part1
        
        # Segments between waypoints
        for i in range(len(waypoints) - 1):
            wp_start = waypoints[i]
            wp_end = waypoints[i + 1]
            
            def real_integrand_wp(t):
                s = wp_start + t * (wp_end - wp_start)
                ds_dt = wp_end - wp_start
                return np.real(func(s) * ds_dt)
            
            def imag_integrand_wp(t):
                s = wp_start + t * (wp_end - wp_start)
                ds_dt = wp_end - wp_start
                return np.imag(func(s) * ds_dt)
            
            real_part_wp, _ = quad(real_integrand_wp, 0, 1, epsabs=abstol, epsrel=reltol)
            imag_part_wp, _ = quad(imag_integrand_wp, 0, 1, epsabs=abstol, epsrel=reltol)
            total_integral += real_part_wp + 1j * imag_part_wp
        
        # Last segment: last waypoint to end
        wp_last = waypoints[-1]
        def real_integrand_last(t):
            s = wp_last + t * (end - wp_last)
            ds_dt = end - wp_last
            return np.real(func(s) * ds_dt)
        
        def imag_integrand_last(t):
            s = wp_last + t * (end - wp_last)
            ds_dt = end - wp_last
            return np.imag(func(s) * ds_dt)
        
        real_part_last, _ = quad(real_integrand_last, 0, 1, epsabs=abstol, epsrel=reltol)
        imag_part_last, _ = quad(imag_integrand_last, 0, 1, epsabs=abstol, epsrel=reltol)
        total_integral += real_part_last + 1j * imag_part_last
        
        return total_integral

In [8]:
def I_Func(an, An, AAn, ap, Ap, AAp, bm, Bm, BBm, bq, Bq, BBq, z, v):
    """
    Exact Python implementation of MATLAB I_Func
    The I-function generalizes the Fox H-function
    """
    # Convert all inputs to numpy arrays, preserving empty arrays exactly as MATLAB
    an = np.array(an, dtype=float, ndmin=1) if np.size(an) > 0 else np.array([])
    An = np.array(An, dtype=float, ndmin=1) if np.size(An) > 0 else np.array([])
    AAn = np.array(AAn, dtype=float, ndmin=1) if np.size(AAn) > 0 else np.array([])
    ap = np.array(ap, dtype=float, ndmin=1) if np.size(ap) > 0 else np.array([])
    Ap = np.array(Ap, dtype=float, ndmin=1) if np.size(Ap) > 0 else np.array([])
    AAp = np.array(AAp, dtype=float, ndmin=1) if np.size(AAp) > 0 else np.array([])
    bm = np.array(bm, dtype=float, ndmin=1) if np.size(bm) > 0 else np.array([])
    Bm = np.array(Bm, dtype=float, ndmin=1) if np.size(Bm) > 0 else np.array([])
    BBm = np.array(BBm, dtype=float, ndmin=1) if np.size(BBm) > 0 else np.array([])
    bq = np.array(bq, dtype=float, ndmin=1) if np.size(bq) > 0 else np.array([])
    Bq = np.array(Bq, dtype=float, ndmin=1) if np.size(Bq) > 0 else np.array([])
    BBq = np.array(BBq, dtype=float, ndmin=1) if np.size(BBq) > 0 else np.array([])
    
    z = complex(z)
    
    # Define F(s) exactly as in MATLAB
    def F(s):
        s = complex(s)
        numerator = (GammaProd(bm, Bm, BBm, s) * 
                    GammaProd(1-an, -An, AAn, s) * 
                    (z ** (-s)))
        denominator = (GammaProd(1-bq, -Bq, BBq, s) * 
                      GammaProd(ap, Ap, AAp, s))
        return numerator / denominator
    
    # Parameters calculation - exact MATLAB behavior
    An_Ap = np.concatenate([An, Ap]) if len(An) > 0 or len(Ap) > 0 else np.array([])
    Bm_Bq = np.concatenate([Bm, Bq]) if len(Bm) > 0 or len(Bq) > 0 else np.array([])
    
    p = len(An_Ap)
    q = len(Bm_Bq)
    
    alphaFox = np.sum(An) - np.sum(Ap) + np.sum(Bm) - np.sum(Bq)
    mu = np.sum(Bm_Bq) - np.sum(An_Ap)
    
    # Calculate betaFox exactly as MATLAB
    betaFox = 1.0
    if len(An_Ap) > 0:
        # Handle case where An_Ap contains zeros
        non_zero_mask = An_Ap != 0
        if np.any(non_zero_mask):
            betaFox *= np.prod((An_Ap[non_zero_mask]) ** (-An_Ap[non_zero_mask]))
    
    if len(Bm_Bq) > 0:
        # Handle case where Bm_Bq contains zeros  
        non_zero_mask = Bm_Bq != 0
        if np.any(non_zero_mask):
            betaFox *= np.prod((Bm_Bq[non_zero_mask]) ** (-Bm_Bq[non_zero_mask]))
    
    # Calculate delta
    bm_bq = np.concatenate([bm, bq]) if len(bm) > 0 or len(bq) > 0 else np.array([])
    an_ap = np.concatenate([an, ap]) if len(an) > 0 or len(ap) > 0 else np.array([])
    
    delta = np.sum(bm_bq) - np.sum(an_ap) + (p - q) / 2
    
    Tol = 1e-5
    
    # Conditions per contour - exact MATLAB logic
    condition01 = alphaFox > 0 and abs(np.angle(z)) < np.pi * alphaFox / 2
    condition02 = (alphaFox == 0 and (delta * mu + np.real(delta)) < -1 and 
                  np.angle(z) == 0)
    condition0 = condition01 or condition02
    
    # contour L_(-infinity)
    condition11 = (mu > 0) and z != 0
    condition12 = (mu == 0) and abs(z) < betaFox and abs(z) > 0
    condition13 = (mu == 0) and abs(z) == betaFox and np.real(delta) < -1
    condition1 = condition11 or condition12 or condition13
    
    # contour L_(+infinity)
    condition21 = (mu < 0) and z != 0
    condition22 = (mu == 0) and abs(z) > betaFox
    condition2 = condition21 or condition22
    
    # Contour preparation - exact MATLAB logic
    epsilon = 10**1.2
    
    # Calculate Sups and Infs exactly as MATLAB
    if len(an) > 0 and len(An) > 0:
        Sups = np.min((v + 1 - an) / An)
    else:
        Sups = None  # equivalent to empty in MATLAB
        
    if len(bm) > 0 and len(Bm) > 0:
        Infs = np.max(-(bm + v) / Bm)
    else:
        Infs = None  # equivalent to empty in MATLAB
    
    # WPx calculation exactly as MATLAB
    if Sups is None and Infs is None:
        WPx = 1
    elif Sups is None and Infs is not None:
        WPx = Infs + epsilon
    elif Sups is not None and Infs is None:
        WPx = Sups - epsilon
    else:
        WPx = (Sups + Infs) / 2
    
    WayPoints = [WPx - 1j * epsilon, WPx + 1j * epsilon]
    
    # Integration - exact MATLAB behavior
    if condition0 or (not condition1 and not condition2):
        infity = 10
        start_point = WPx - 1j * infity
        end_point = WPx + 1j * infity
        
        integral_result = complex_contour_integral(F, start_point, end_point, 
                                                 abstol=Tol, reltol=Tol)
        out = np.real((1 / (2j * np.pi)) * integral_result)
        return out
    
    if condition1:
        infity = 100
        if len(bm) > 0 and len(Bm) > 0:
            min_val = np.min(-bm / Bm)
            infity = infity - min_val
        
        # MATLAB: integral(F, -infity, -infity, 'Waypoints', WayPoints)
        # This integrates from -infity to -infity via waypoints (closed contour)
        start_point = -infity + 0j
        end_point = -infity + 0j
        
        integral_result = complex_contour_integral(F, start_point, end_point, 
                                                 waypoints=WayPoints)
        out = np.real((1 / (2j * np.pi)) * integral_result)
        return out
    
    if condition2:
        infity = 100
        if len(an) > 0 and len(An) > 0:
            max_val = np.max((1 - an) / An)
            infity = infity + max_val
        
        # MATLAB: integral(F, infity, infity, 'Waypoints', WayPoints, ...)
        # This integrates from infity to infity via waypoints (closed contour)
        start_point = infity + 0j
        end_point = infity + 0j
        
        integral_result = complex_contour_integral(F, start_point, end_point, 
                                                 waypoints=WayPoints, 
                                                 abstol=Tol, reltol=Tol)
        out = np.real((1 / (2j * np.pi)) * integral_result)
        return out

In [9]:
def I_for_exact_param(gamma_v, k, j, v=1):
    
    rho = 3.3
    K = 10
    sigma_squared = 1/(1+K)
    A0 = 1
    A0_squared = A0 ** 2
    h_i = 0.4993
    h_i_squared = h_i**2
    gamma_bar = 1
    base_power = 2 * sigma_squared * A0_squared * h_i_squared * gamma_bar
    z =  gamma_v/base_power

    # Scalars
    an = np.array(1 - (rho**2 / 2))
    An = np.array(1)
    AAn = np.array(1)

    # Vectors (lists)
    ap = np.array([1 + z - (rho**2), 1])
    Ap = np.array([z, 1])
    AAp = np.array([k, 1])

    bm = np.array([j - (rho**2 / 2) + 1, 0, z - rho**2])
    Bm = np.array([1, 1, 2])
    BBm = np.array([1, 1, k])

    bq = np.array(- (rho**2 / 2))
    Bq = np.array(1)
    BBq = np.array(1)

    return I_Func(an, An, AAn, ap, Ap, AAp, bm, Bm, BBm, bq, Bq, BBq, z, v)

In [10]:
def F_gamma(gamma_v, gamma_bar):
    """
    Calculate F_gamma function based on the mathematical expression.
    
    Parameters:
    gamma_v: gamma value
    gamma_bar: gamma bar value
    
    Returns:
    float: Result of F_gamma calculation
    """
    K = 10
    m = 2.3
    Delta = 0.01
    n_max = 30
    
    # Start with front expression
    result = front_expression(gamma_v, gamma_bar)
    
    # Add summation term
    summation = 0
    for j in range(n_max + 1):  # 0 to 30 inclusive
        term = (a_j(j, m, Delta) * (K**j) * I_for_exact_param(gamma_v, K, j)) / ((factorial(j))**2)
        summation += term
    
    result *= summation
    
    return result

In [11]:
print(F_gamma(1,10))

/tmp/ipykernel_54098/1513950100.py:24: RuntimeWarning: overflow encountered in power
  gamma_vals = gammaz(gamma_args) ** y
/tmp/ipykernel_54098/1513950100.py:24: RuntimeWarning: invalid value encountered in power
  gamma_vals = gammaz(gamma_args) ** y
/home/raw/Documents/workspace/codes/FogWireless_CDF_Analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: cannot reshape array of size 3 into shape ()